# Nettoyage du text

Etapes de néttoyage:
1. Mettre en minuscle le texte
2. Enlever les contractions
3. Enlever les espaces
4. Tokeniser le texte
5. Lemmatiser les mots
6. Enlever les éléments ininteréssant (chiffres, ponctuation, mots non anglais et lettres isolées)

In [23]:
import pickle
import re
import string
import warnings

import matplotlib.pyplot as plt
import nltk
import pandas as pd
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn import metrics


warnings.filterwarnings("ignore")

stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['one', 'two', 'three', 'four', 'five', 'six', 'seven',
                   'eight', 'nine', 'go', 'goes', 'get', 'also', 'however', 'tells']
stopwords_list += [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
stopwords_list += [x for x in string.ascii_lowercase]


def clean_text(text: str):
    text = str(text)
    text = text.lower()

    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('\d+', ' ', text)

    text = text.strip(' ')
    text = nltk.word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    stem_sentence = []
    for word in text:
        stem = lemmatizer.lemmatize(word)
        stem_sentence.append(stem)

    text = [w for w in stem_sentence if w not in stopwords_list]
    text = ' '.join(text)
    return text


if __name__ == "__main__":
    # to_csv("archive/dataset/test_data.txt", "test_data_clean.csv")
    # to_csv_train_solution("archive/dataset/test_data_solution.txt", "test_data_solution_clean.csv")
    with open("text.txt", "r", encoding="utf-8") as f:
        text = " ".join(f.readlines())
    print(text)
    print("-----------------------------------------------------")
    cleaned_text = clean_text(text)
    print(cleaned_text)


Dune
 The story of Paul Atreides, a young man as gifted as he was brilliant, destined to meet an extraordinary destiny that totally surpasses him. Because if he wants to preserve the future of his family and his people, he will have to go to the most dangerous planet in the universe - the only one able to provide the most precious resource in the world, capable of multiplying tenfold. power of mankind. As evil forces vie for control of this planet, only those who manage to overcome their fear will be able to survive ...
-----------------------------------------------------
dune story paul atreides young man gifted wa brilliant destined meet extraordinary destiny totally surpasses want preserve future family people dangerous planet universe able provide precious resource world capable multiplying tenfold power mankind evil force vie control planet manage overcome fear able survive


# Nettoyage de la base de donnée

L'une des première étapes étant de nettoyer notre base de données.

Pour se faire nous avons transformer la base de donnée de base en txt vers csv tout en nettoyant le text avec la fonction clean_text pour que ce soit plus facile à manipuler.

**Base de donnée txt:**

<img src="img/clean_db/db_txt.PNG">

**Base de donnée csv nettoyée:**

<img src="img/clean_db/db_csv_clean.PNG">

In [24]:
def to_csv(file, output):
    with open(file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    df = pd.DataFrame(columns=["id", "title", "description"])

    for i in range(len(lines)):
        line = lines[i].split(":::")

        df_tmp = pd.DataFrame([(int(line[0]), line[1], clean_text(line[2]))], columns=[
                              "id", "title", "description"])
        df = df.append(df_tmp)
    df.to_csv(output)
    print(output, "Saved")


def to_csv_train_solution(file, output):
    with open(file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    df = pd.DataFrame(columns=["id", "title", "genre", "description"])

    for i in range(len(lines)):
        line = lines[i].split(":::")

        df_tmp = pd.DataFrame([(int(line[0]), line[1], line[2], clean_text(
            line[3]))], columns=["id", "title", "genre", "description"])
        df = df.append(df_tmp)
    df.to_csv(output)
    print(output, "Saved")


# Fréquence des mots

En regardant la fréquence des mots par genre nous avons pu remarquer qu’il varié selon ce dernier.

Ça nous a donc donné notre première idée de classifieur, la classification par champs lexical.

Exemples de fréquence de mot par genre:

Drama :

<img src="img/freq_word/drama%20.png">

Comedy :

<img src="img/freq_word/comedy%20.png">

Horror :

<img src="img/freq_word/horror%20.png">


Si vous voullez voir les mots les plus fréquents par genre, vous pouvez aller dans le dossier img/freq_word.

In [25]:
def word_frequeny(df):
    texts_genre = {}

    for i in df.index:
        genre = df["genre"][i]
        if genre not in texts_genre:
            texts_genre[genre] = df["description"][i]
        else:
            texts_genre[genre] += " "
            texts_genre[genre] += df["description"][i]

    for text in texts_genre.values():

        data_analysis = nltk.FreqDist(text.split(" "))

        filter_words = dict([(m, n)
                            for m, n in data_analysis.items() if len(m) > 3])

        data_analysis = nltk.FreqDist(filter_words)
        
        data_analysis.plot(10, cumulative=False)


# Première idée de classifieur (champ lexical)

Après avoir calculer la fréquence des mots par genre, nous nous sommes donc orienté vers une classification par rapport au champ lexical.

Ce classifieur nous a induit a compté le nombre d'apparitions de mots par genre et ensuite déterminé les mots qui apparaissent le plus par genre.

In [26]:
import re
import pandas as pd
from tqdm import trange


def clean_lexical(df):
    lexical = df["lexical_field"]
    # new_df = pd.DataFrame()
    for i in range(len(df)):
        splited = lexical[i].split(" ")
        l = clean_text(" ".join(set(splited)))
        df["lexical_field"][i] = l

    df.to_csv("archive/lexical_field_clean.csv")


def similar(summary, lexical_field):
    summary = summary.split(" ")
    lexical_field = lexical_field.split(" ")

    ret = 0
    for s in summary:
        if s in lexical_field:
            ret += 1
    return ret

# partially working version


def get_similarities_counter(df, text):
    lexical = df["lexical_field"]

    genre = ""
    note = 0
    clean_test = clean_text(text)
    for i in range(len(df)):
        tmp = similar(clean_test, lexical[i])
        if note < tmp:
            note = tmp
            genre = df["genre"][i]

    # print(genre)
    return genre

# df = pd.read_csv("./archive/lexical_field_clean.csv")
# clean_lexical(df)
# data = pd.read_csv("./archive/dataset_csv/test_data_solution.csv")

# sentences = data["description"].values

# genrePredit=""
# error=0

# for i in trange(len(sentences)):
#     genrePredit=get_similarities_counter(df, sentences[i])
#     if genrePredit!=data["genre"][i]:
#         error+=1
#         # print(genrePredit+" "+data["genre"][i])

# error=(error/len(sentences))*100
# print(error)
# if __name__ == "__main__":
#     with open("text.txt", "r", encoding="utf-8") as f:
#         text = " ".join(f.readlines())

#     df = pd.read_csv("archive/dataset_csv/test_data_solution_clean.csv")

#     word_frequeny(df)


# TrainClassifier

Cette class nous permet d'entrainer un classifieur sur un corpus de données.

Nous pouvons choisir le modèle de classifieur à utiliser, le corpus de données à utiliser, le nombre de classes à distinguer, le nombre de fois que l'on souhaite entrainer le modèle, le nombre de fois que l'on souhaite tester le modèle et le nombre de fois que l'on souhaite évaluer le modèle.

In [27]:

class TrainClassifier(object):
    def __init__(self, df_path="archive/dataset_csv/train_data_clean.csv", test_size=0.1, train_size=0.5, sampling=None):
        self.vectorizer = None
        self.df = pd.read_csv(df_path)
        self.classifier = None
        self.vectorizer = TfidfVectorizer()
        self.test_size = test_size
        self.train_size = train_size
        self.sentences = self.df["description"].values
        self.Y = self.df["genre"].values
        self.sampling = sampling if sampling is not None else None

    def train(self, classifier, model_path="neural_network_model.sav", **kwargs):
        print(f"Start training with {classifier}...")

        if self.sampling is not None:
            self.sentences = shuffle(self.sentences, n_samples=self.sampling)
            self.Y = shuffle(self.Y, n_samples=self.sampling)

        self.vectorizer = TfidfVectorizer()
        self.vectorizer.fit(self.sentences)
        self.vectorizer.transform(self.sentences)

        sentences_train, sentences_test, y_train, y_test = train_test_split(
            self.sentences, self.Y, test_size=self.test_size, train_size=self.train_size)  # , random_state=50
        self.vectorizer = TfidfVectorizer()
        self.vectorizer.fit(sentences_train)

        X_train = self.vectorizer.transform(sentences_train)
        X_test = self.vectorizer.transform(sentences_test)

        self.classifier = classifier(**kwargs)

        self.classifier.fit(X_train, y_train)

        save_model(dict(classifier=self.classifier, vectorizer=self.vectorizer),
                   model_path)

        score = self.classifier.score(X_test, y_test)
        print(f"Trained with {score:.3f}% accuracy")
        metrics.ConfusionMatrixDisplay.from_estimator(
            self.classifier, X_test, y_test, labels=self.df["genre"].unique(), include_values=False, xticks_rotation=90)
        plt.show()
        plt.clf()
        return score


# Sauvegarder et charger un modèle pré-entrainé

Ces fonctions nous permettent de charger un modèle ou de sauvegardé un modèle.
Grace à ces fonctions nous ne sommes pas obligé d’entrainer notre réseau de neurones à chaque utilisation, il nous suffit de charger celui qui a était généré préalablement. 

Charger un modèle permet également de garantir le meilleur résultat que nous ayons trouver pour se classifieur.

In [28]:
def save_model(model, file_path):
    pickle.dump(model, open(file_path, "wb+"))
    print(f"{file_path} saved")


def load_model(file_path):
    model = pickle.load(open(file_path, "rb"))
    print(f"{model} loaded")
    print(model["vectorizer"])
    return model


# Tests du classifieur par SVM

In [29]:


def graph_train_svc(df_trained_data_path, df_to_predict):
    test_sizes = [x/10 for x in range(1, 10)]
    train_sizes = [x/10 for x in range(1, 10)]

    x_labels = [0]
    x_ticks = [0]
    i = 0
    for a in test_sizes:
        for b in train_sizes:
            x_labels.append("("+str(a)+","+str(b)+")")
            i += 1
            x_ticks.append(i)
    i = 1

    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
    plt.xlabel("(test_size, train_size)")
    plt.ylabel("Percentage (%)")
    plt.title("SVM variation")
    plt.grid(True)
    x = []
    y_accuracy = []
    y_error = []

    nbOfGraph = 0
    for test_size in test_sizes:
        for train_size in train_sizes:
            if test_size+train_size < 1:
                print(str(test_size)+" "+str(train_size))
                print(x_ticks[i])
                print(x_labels[i])

                trainer = TrainClassifier(
                    df_trained_data_path, test_size, train_size)
                score = trainer.train(
                    classifier=LinearSVC, model_path="svm_model.sav")

                predict("predicted_svm.csv",
                        df_to_predict, classifier_method="svm")
                error = predictError(pd.read_csv(
                    "predicted_svm.csv"))
                y_error.append(error)

                y_accuracy.append(score*100)
                x.append(x_ticks[i])
                print("Accuracy:", score)

            if i != 0 and i % 9 == 0:
                plt.scatter(x, y_accuracy)
                plt.scatter(x, y_error)
                plt.legend(["Accuracy", "Error"])

                plt.savefig(
                    f"svm-variation-{nbOfGraph}.png")
                y_accuracy = []
                y_error = []
                x = []
                plt.clf()
                plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
                plt.xlabel("(test_size, train_size)")
                plt.ylabel("Percentage (%)")
                plt.title("SVM variation")
                plt.grid(True)

                nbOfGraph += 1
            i += 1


# Tests du classifieur par LogisticRegression

Ici nous testons différents paramètres de notre classifieur, notamment le nombre de classes à distinguer, le nombre de fois que l'on souhaite entrainer le modèle, le nombre de fois que l'on souhaite tester le modèle et le nombre de fois que l'on souhaite évaluer le modèle.

Enfin, nous affichons les résultats de notre classifieur à l'aide de graphs pour détecter les paramètres qui ont le meilleur score.

Voici le meilleur résultat obtenu parmis ceux réalisé par nos tests: (test_size=0.2, train_size=0.7) avec une accuracy de 58% et un taux d'erreur d'environ 42%.

<img src="graph_tests/logistic_regression/variation-error_accuracy-1.png">

Si vous voulez consultez les autres tests, vous pouvez aller dans le dossier graph_tests/logistic_regression/

In [30]:

def graph_train_naif_logistic_regression(df_trained_data_path, df_to_predict):
    test_sizes = [x/10 for x in range(1, 10)]
    train_sizes = [x/10 for x in range(1, 10)]

    x_labels = [0]
    x_ticks = [0]
    i = 0
    for a in test_sizes:
        for b in train_sizes:
            x_labels.append("("+str(a)+","+str(b)+")")
            i += 1
            x_ticks.append(i)
    i = 1

    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
    plt.xlabel("(test_size, train_size)")
    plt.ylabel("Percentage (%)")
    plt.title("Naïf Logistic Regression variation")
    plt.grid(True)
    x = []
    y_accuracy = []
    y_error = []

    nbOfGraph = 0
    for test_size in test_sizes:
        for train_size in train_sizes:
            if test_size+train_size < 1:
                print(str(test_size)+" "+str(train_size))
                print(x_ticks[i])
                print(x_labels[i])

                trainer = TrainClassifier(
                    df_trained_data_path, test_size, train_size)
                score = trainer.train(
                    classifier=LogisticRegression, model_path="naif_logistic_regression_model.sav")

                predict("predicted_logistic_regression.csv",
                        df_to_predict, classifier_method="logistic_regression")
                error = predictError(pd.read_csv(
                    "predicted_logistic_regression.csv"))
                y_error.append(error)

                y_accuracy.append(score*100)
                x.append(x_ticks[i])
                print("Accuracy:", score)

            if i != 0 and i % 9 == 0:
                plt.scatter(x, y_accuracy)
                plt.scatter(x, y_error)
                plt.legend(["Accuracy", "Error"])

                plt.savefig(
                    f"variation-error_accuracy-{nbOfGraph}.png")
                y_accuracy = []
                y_error = []
                x = []
                plt.clf()
                plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
                plt.xlabel("(test_size, train_size)")
                plt.ylabel("Percentage (%)")
                plt.title("Naïf Logistic Regression variation")
                plt.grid(True)

                nbOfGraph += 1
            i += 1


# Tests du classifieur par MLPClassifier

Ici nous testons différents paramètres de notre classifieur, notamment le nombre de layers et le nombre de neurones par layer.

Enfin, nous affichons les résultats de notre classifieur à l'aide de graphs pour détecter les paramètres qui ont le meilleur score.

Voici le meilleur résultat obtenu parmis ceux réalisé par nos tests: (layers=1, neurones=100) avec une accuracy de 58% et un taux d'erreur d'environ 42%.

<img src="./graph_tests/mlp_classifier/neural_network_variation-error_accuracy-0.png">

Vous pouvez aussi consultez les autres tests, vous pouvez aller dans le dossier graph_tests/mlp_classifier/

In [31]:


def graph_train_neural_network(df_trained_data_path, df_to_predict):
    x_labels = [0]
    x_ticks = [0]
    i = 0
    for layer in range(1, 5):
        for neuron in range(1, 101, 9):
            x_labels.append(str(layer)+" x "+str(neuron))
            i += 1
            x_ticks.append(i)
    i = 1

    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
    plt.xlabel("layer X neurons")
    plt.ylabel("Percentage (%)")
    plt.title("Neural Network MLPClassifier variation")
    plt.grid(True)

    x = []
    y_accuracy = []
    y_error = []

    nbOfGraph = 0
    for layer in range(1, 5):
        for neuron in range(1, 101, 9):
            print(str(layer)+" x "+str(neuron))
            print(x_ticks[i])
            print(x_labels[i])
            trainer = TrainClassifier(
                df_trained_data_path, 0.2, 0.7)
            score = trainer.train(classifier=MLPClassifier,
                                  hidden_layer_sizes=tuple((neuron for i in range(layer))))

            predict("predicted_neural_network.csv",
                    df_to_predict, classifier_method="neural_network")
            error = predictError(pd.read_csv(
                "predicted_neural_network.csv"))

            y_error.append(error)
            y_accuracy.append(score*100)
            x.append(x_ticks[i])

            print("Accuracy:", score)
            i += 1

        plt.scatter(x, y_accuracy)
        plt.scatter(x, y_error)
        plt.legend(["Accuracy", "Error"])
        plt.savefig(
            f"neural_network_variation-error_accuracy-{nbOfGraph}.png")
        y_accuracy = []
        y_error = []
        x = []
        plt.clf()

        plt.xticks(ticks=x_ticks, labels=x_labels, rotation=10)
        plt.xlabel("layer X neurons")
        plt.ylabel("Percentage (%)")
        plt.title("Neural Network MLPClassifier variation")
        plt.grid(True)

        nbOfGraph += 1


# Prédiction 

Pour prédire le genre d’un film nous commençons par récupérer un modèle entrainé, nous vectorisons la description puis nous appelons la fonction predict du classifieur choisie.
Nous pouvons ainsi calculer le taux d’erreur grâce à predictError.

La fonction convert_string_to_dataset_prediction quant à elle sert uniquement à générer un CSV qui pourra être traité par la suite.

In [32]:
def predictError(df):
    genre = df["genre"]
    genre_predicted = df["genre_predit"]
    length_genre = len(genre)
    error = 0
    for i in range(length_genre):
        if genre[i] != genre_predicted[i]:
            error += 1

    print(f"Error : {(error/length_genre)*100:.3f}%")
    return (error/length_genre)*100


def predict(output, df_to_predict, classifier_method="neural_network", sampling=None, model_path="neural_network_model.sav", **kwargs):

    try:
        model = load_model(model_path)
    except:
        trainer = TrainClassifier(sampling=sampling)
        if classifier_method == "neural_network":
            trainer.train(MLPClassifier, **kwargs)
        elif classifier_method == "logistic_regression":
            trainer.train(LogisticRegression, **kwargs)
        elif classifier_method == "svm":
            trainer.train(LinearSVC, **kwargs)

        model = load_model(model_path)

    classifier = model["classifier"]
    vectorizer = model["vectorizer"]
    print(f"Predicting with {classifier}...")

    X_test_to_predict = vectorizer.transform(df_to_predict["description"])

    classified = classifier.predict(X_test_to_predict)
    try:
        # calcul du pourcentage d'erreurs
        df2 = pd.DataFrame(data={
            "title": df_to_predict["title"].values, "description": df_to_predict["description"], "genre": df_to_predict["genre"], "genre_predit": classified})
    except:
        df2 = pd.DataFrame(data={
            "title": df_to_predict["title"].values, "description": df_to_predict["description"], "genre_predit": classified})

    # predictError(df2)
    df2.to_csv(output)
    print(f"Prediction done and saved in {output}")
    print(f"Classifier {classifier} prediction for {df_to_predict['title'].values} is {classified}")


def convert_string_to_dataset_prediction(title, description):
    # print(clean_text(str(description)))
    return pd.DataFrame({"title": str(title), "description": str(clean_text(description))}, index=[0])


def read_text(path):
    with open(path, "r") as f:
        title, description = map(str.strip, f.readlines())
        return title, description


# Main

Le main est la fonction principal du code. Celle qui va lancer les autres fonctions afin de prédire le genre d’un film grâce à son résumé.
C’est également ici qu’on parse les arguments afin de pouvoir personnaliser son éxpérience.

In [33]:
if __name__ == "__main__":
    df_to_predict = convert_string_to_dataset_prediction("Dune", "The story of Paul Atreides, a young man as gifted as he was brilliant, destined to meet an extraordinary destiny that totally surpasses him. Because if he wants to preserve the future of his family and his people, he will have to go to the most dangerous planet in the universe - the only one able to provide the most precious resource in the world, capable of multiplying tenfold. power of mankind. As evil forces vie for control of this planet, only those who manage to overcome their fear will be able to survive ...")
    output = "./predictions/dune.csv"
    predict(output, df_to_predict=df_to_predict,
            classifier_method="svm", model_path="./models/svm.sav")


{'classifier': LinearSVC(), 'vectorizer': TfidfVectorizer()} loaded
TfidfVectorizer()
Predicting with LinearSVC()...
Prediction done and saved in ./predictions/dune.csv
Classifier LinearSVC() prediction for ['Dune'] is [' sci-fi ']
